In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymatreader

In [3]:
import pandas as pd
import numpy as np
from pymatreader import read_mat
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import os

#data_path = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/gait-analysis-main_VT/AI_Frailty-MATLAB/AI_Frailty-MATLAB'

data_path = "/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample/Raw"

In [30]:



def kinz_time(kinz_data):
    frames_num = len(kinz_data)
    time_in_sec = np.zeros(frames_num)
    time0 = kinz_data[0]  # time zero for the acquisition

    for i in range(frames_num):
        time_dbl = kinz_data[i] - time0  # relative time (double precision)
        time_in_sec[i] = time_dbl / 1e9  # Convert time to seconds.

    mean_time_diff = np.mean(np.diff(time_in_sec))  # Mean time difference
    fs = 1 / mean_time_diff  # Sampling rate

    return time_in_sec, fs




def KINZ_DATA_np(k):

  details = os.path.basename(k)

  p_id= details.split("_")[0].split("-")[-2]+"-"+ details.split("_")[0].split("-")[-1]
  traial = details.split("_")[1]
  walking_speed = details.split("_")[2]

  kinzData = read_mat(k)

  footfall_event_times = kinzData['footfall_event_times']

  kinz_arr = np.empty([0,108])


  for i in range(len(kinzData["joint_positions"])):
      timeStamps = np.array(kinzData["t_uniform"][i])
      kinz_x = kinzData["joint_positions"][i][0]
      kinz_y = kinzData["joint_positions"][i][1]
      kinz_z = kinzData["joint_positions"][i][2]
      accel_energy_total = np.array(kinzData['accel_energy_total'][i])
      acc1 = np.array(kinzData['accel_energy'][i][0])
      acc2 = np.array(kinzData['accel_energy'][i][1])
      acc3 = np.array(kinzData['accel_energy'][i][2])
      acc4 = np.array(kinzData['accel_energy'][i][3])
      acc5 = np.array(kinzData['accel_energy'][i][4])
      acc6 = np.array(kinzData['accel_energy'][i][5])


      if timeStamps in footfall_event_times:
        fet = np.array(1)
      else:
        fet = np.array(0)

      d = np.concatenate((p_id,traial,walking_speed,timeStamps,acc1,acc2,acc3,acc4,acc5,acc6,accel_energy_total,fet,kinz_x,kinz_y,kinz_z), axis=None)

      kinz_arr = np.append(kinz_arr, [d], axis=0)

  return kinz_arr




In [31]:



final = np.empty([0,108])

for i in tqdm(sorted(glob(data_path+"/*"))):
  for k in sorted(glob(i+"/TS*.mat")):
    print("Processing: ", os.path.basename(k))
    temp_single = KINZ_DATA_np(k)
    #time_elements = temp_single[:, 3]

    #time_elements = time_elements.astype(float)
    #replacement_values = kinz_time(time_elements)[0]

    #temp_single[:, 3] = replacement_values
    final = np.append(final, temp_single, axis=0)





joints = [
    'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
    'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
    'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
    'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
    'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
    'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]


all_columns = ['patientID','trial_num','walking_speed','timeStamps','accel_energy_1','accel_energy_2','accel_energy_3','accel_energy_4','accel_energy_5','accel_energy_6','accel_energy_total','footfall_event_times']


for joint in joints:
  x = f'{joint}_X'
  all_columns.append(x)

for joint in joints:
  y = f'{joint}_Y'
  all_columns.append(y)

for joint in joints:
  z = f'{joint}_Z'
  all_columns.append(z)




final_df = pd.DataFrame(final, columns=all_columns)

columns_to_convert = final_df.columns.difference(['patientID','trial_num','walking_speed'])
final_df[columns_to_convert] = final_df[columns_to_convert].astype(float)




  0%|          | 0/4 [00:00<?, ?it/s]

Processing:  TS-Cln-001-LO_Trial1_Regular_20220307_111113.mat
Processing:  TS-Cln-001-LO_Trial2_Regular_20220307_111139.mat
Processing:  TS-Cln-001-LO_Trial3_Fast_20220307_111208.mat
Processing:  TS-Cln-001-LO_Trial4_Fast_20220307_111234.mat
Processing:  TS-Cln-001-LO_Trial5_Fast_20220307_111347.mat
Processing:  TS-Cln-001-LO_Trial6_Fast_20220307_111427.mat
Processing:  TS-Cln-001-LO_Trial6_Fast_20220307_111722.mat
Processing:  TS-Cln-002-RJ_Trial2_Regular_20220311_100440.mat
Processing:  TS-Cln-002-RJ_Trial3_Fast_20220311_100509.mat
Processing:  TS-Cln-002-RJ_Trial4_Fast_20220311_100541.mat
Processing:  TS-Cln-002-RJ_Trial4_Fast_20220311_100659.mat
Processing:  TS-Cln-RM-003_Trial1_Regular_20220315_060038.mat
Processing:  TS-Cln-RM-003_Trial2_Regular_20220315_060059.mat
Processing:  TS-Cln-RM-003_Trial3_Fast_20220315_060209.mat
Processing:  TS-Cln-RM-003_Trial4_Fast_20220315_060226.mat
Processing:  TS-Cln-RM-003_Trial4_Fast_20220315_060742.mat
Processing:  TS-Cln-004-MF_Trial3_Regular

In [35]:
saving_path="/content/drive/MyDrive/Iqram Sir/AI_Frailty-MATLAB_Sample"
final_df.to_csv(saving_path+'/KINZ_JOINT_with_acc.csv', index=False)